In [1]:
import datetime
import json
import numpy as np
import pandas as pd
import random
import secrets

# Load real impact attestations

In [2]:
json_path = 'data/attestations.json'
with open(json_path, 'r') as f:
    data = json.load(f)
print(len(data))    

149


In [3]:
relevant_attestations = []
for a in data:
    qs = a.get('metadata', {}).get('impactAttestations', [])
    
    if len(qs) < 2:
        continue
    
    nps = pmf = None
    for q in qs:
        val = q.get('value')
        if not val:
            continue
        if q['name'] == 'Likely to Recommend':
            nps = val
        elif q['name'] == 'Feeling if didnt exist':
            pmf = val
    
    if not (nps or pmf):
        continue
        
    relevant_attestations.append({
        'id': a['id'],
        'attester': a['attester'],
        'recipient': a['recipient'],
        'farcasterID': a['farcasterID'],
        'projectRegUID': a['projectRegUID'],
        'timeCreated': datetime.datetime.utcfromtimestamp(a['timeCreated']),
        'issuer': a['issuer'],
        'metadataurl': a['metadataurl'],
        'nps_score': pd.to_numeric(nps),
        'pmf_score': pd.to_numeric(pmf)
    })

In [4]:
df_attestations_actual = pd.DataFrame(relevant_attestations)
df_attestations_actual.tail(1).T

,15
id,0xecc8d17aa547682ffc8b4b69c1ed59bd642fad4dc6c8...
attester,0x7484aABFef9f39464F332e632047983b67571C0a
recipient,0x5d36a202687fD6Bd0f670545334bF0B4827Cc1E2
farcasterID,"{'type': 'BigNumber', 'hex': '0x2d4c'}"
projectRegUID,0x82169691e4026be1d950f20c2d36500b06fb6f3e0c03...
timeCreated,2024-10-02 06:16:34
issuer,MGL
metadataurl,https://gateway.pinata.cloud/ipfs/QmeYyTJzXsXS...
nps_score,6
pmf_score,2.0


# Create some dummy data attestations

In [5]:
NUM_PROJECTS = 30
NUM_ATTESTERS = 100
NUM_ATTESTATIONS = 1000
PMF_SCALE = 5
NPS_SCALE = 10

COUNCILS = ['Anticapture Commission', 'Code of Conduct Council', 'Collective Feedback Committee',
            'Developer Advisory Board', 'Grants Council', 'Security Council']

projects = {
    f"0x{secrets.token_hex(32)}": random.random()
    for _ in range(NUM_PROJECTS)
}

attesters = {}
for a in range(NUM_ATTESTERS):
    addr = f"0x{secrets.token_hex(20)}"
    
    is_citizen = random.randint(0,10) < 6
    is_delegate = random.randint(0,10) < 3
    councils = set()
    if is_citizen or is_delegate:
        for _ in range(2):
            idx = random.randint(0,10)
            if idx < len(COUNCILS):
                councils.add(COUNCILS[idx])
    attesters.update({
        addr:{
            'is_citizen': is_citizen,
            'is_top_delegate': is_delegate,
            'governance_membership': list(councils)
        }
    })

attestations = []
for i in range(NUM_ATTESTATIONS):
    random_project = random.choice(list(projects.keys()))
    random_attester = random.choice(list(attesters.keys()))
    project_impact = projects.get(random_project)
    pmf_score = random.randint(int(project_impact*PMF_SCALE), PMF_SCALE)
    nps_score = random.randint(int(project_impact*NPS_SCALE), NPS_SCALE)
    attestations.append({
        'uid': f"0x{secrets.token_hex(32)}",
        'attester': random_attester,
        'rf6_applicationUID': random_project,
        'pmf_score': max(1,pmf_score),
        'nps_score': max(1,nps_score),
        **attesters[random_attester]
    })

In [6]:
with open("data/dummy_attestations.json", "w") as f:
    json.dump(attestations,f,indent=2)

attestations[0]

{'uid': '0x0c845fd444022ef274699b918996c1e3a3ad7887fd92083646bce3df9d7a4922',
 'attester': '0x13a5995756acf96e62b80009d37538f140370e37',
 'rf6_applicationUID': '0x59ccc24d38de83c3db70f5326d3d12ddcbca3fc9eb1eeb3df57322f730662f2f',
 'pmf_score': 2,
 'nps_score': 1,
 'is_citizen': True,
 'is_top_delegate': False,
 'governance_membership': ['Anticapture Commission',
  'Developer Advisory Board']}

# Implement a few of the metrics

1. Number of Attestations (only want the count from Citizens and Top Delegates)
2. Number of Attestations by Citizens
3. Number of Attestations by Top delegates
4. Average NPS score of Citizens and Top delegates
5. Most positive superlative
6. "Can't live without" superlative
7. Percentage distribution of different ratings by citizens and top delegates [Extremely upset, Somewhat upset, Neutral]
    1. Percentage distribution of different ratings by citizens
    2. Percentage distribution of different ratings by delegates
8. Number of elected governance members that attested
    1. Number of each group of elected governance members that attested
    2. Average NPS score per elected governance group
    3. Average feeling per elected governance group

In [7]:
df_sim = pd.DataFrame(attestations)
df_sim.tail(1)

,uid,attester,rf6_applicationUID,pmf_score,nps_score,is_citizen,is_top_delegate,governance_membership
999,0xff064eefee632c3aa0476cb57407f45e8dd55597bcd5...,0x8e13c224a18ae97a6ecfa64d3c1313c93970f5b0,0xfb558086c0823285b276ce47e53b7628536f94f99759...,4,3,True,False,[Grants Council]


In [8]:
count_total_attestations = df_sim.groupby('rf6_applicationUID')['uid'].count().rename('count_total_attestations')
count_citizen_attestations = df_sim[df_sim.is_citizen].groupby('rf6_applicationUID')['uid'].count().rename('count_citizen_attestations')
count_delegate_attestations = df_sim[df_sim.is_top_delegate].groupby('rf6_applicationUID')['uid'].count().rename('count_delegate_attestations')
avg_nps_score = df_sim.groupby('rf6_applicationUID')['nps_score'].mean().rename('avg_nps_score')
avg_nps_score

rf6_applicationUID
0x05ff156dd52db407d3857d3547cf3e41129ad6bc59ce123a75ec6342e7719d97    7.076923
0x0ea28b549e68cb209e3dbb5a6fd7cb8bb4c0f5b67c1d57daa7502252840a7621    8.368421
0x1373963ae5eb556c030a63b258412a32cac7bd3f22c9816b4715f035e010f7e9    7.147059
0x1457d66e809cab15ba9fcf7c88aa77c3315d9791e7207a11e7590c4e9f9d37f3    7.103448
0x1e4c7fe036325be4fceb6e6b72f7def4bf8903ff93a6db551c295d155f78778e    7.861111
0x22e9d4cda0e51827329bc47034d2b5e207295eeab74ac8c9dfc499696e5d9e47    7.085714
0x2fa20eecf487ae5880a905e0703fe2d3b101c28d76724f34243b7149e56ef0f0    6.696970
0x33a567a56f28e046f2bc1aa179edf1fec1cc345bac657e6d258e259d6dda6ef5    7.027778
0x3b2a0ae3c6fe60df4d5891f29cc16d592f4801243e32de69297ab7a0a44567a3    6.241379
0x3bea3a266677fdad230f7d25443fb8ae1cf70e3c6680e70d9d17b485c69dc9b2    5.576923
0x3e2c1e257a7f54945d1ebd8a1393d910085e95e0f66e7c2c96b3c788d45eaf43    5.529412
0x4735271db4c5d0bad6553f3c5e0ab9c12be1e15c7577be059b05e1f807c83948    6.343750
0x4d818dfe170046b0cb16f2534ae211c